In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import math
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import os
import itertools
import numpy as np
import pandas as pd

In [4]:
class TextValueDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, dial_col):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.dial_col = dial_col

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row[self.dial_col]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

def create_dataloader(df, tokenizer, dial_col, max_length=256, batch_size=16):
    dataset = TextValueDataset(df, tokenizer, max_length, dial_col)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

def split_merge(df, sp_col):
    cols = list(df.columns)
    cols = cols[:3] + [col + '_' + sp_col for col in cols[3:]]
    col_map = {list(df.columns)[i]:cols[i] for i in range(len(cols))}
    df = df.rename(columns=col_map)
    return df

def remake_df(df, appdf, sp1col, sp2col):
    sp1_df = appdf[appdf['indices'] == 1]
    sp2_df = appdf[appdf['indices'] == 0]
    sp1_df.rename(columns={"utterances": sp1col}, inplace=True)
    sp2_df.rename(columns={"utterances": sp2col}, inplace=True)
    sp1_df.reset_index(inplace=True)
    sp2_df.reset_index(inplace=True)
    sp1_df = split_merge(sp1_df, sp1col)
    sp2_df = split_merge(sp2_df, sp2col)
    df = pd.concat([df, sp1_df], axis=1)
    df = pd.concat([df, sp2_df], axis=1)
    return df

In [7]:
aloe_df = pd.read_csv('./appraised_dialogues/google-t5_ALOE_train.csv')
aloe_df

,Unnamed: 0,id,subreddit,parent_id,target_id,target_text,observer_id,observer_text,full_text,distress_score,...,accept_conseq_observer_text,predict_event_observer_text,urgency_observer_text,other_responsblt_observer_text,self_control_observer_text,chance_control_observer_text,standards_observer_text,goal_relevance_observer_text,attention_observer_text,unpleasantness_observer_text
0,0,1427,abusiverelationships,t3_o71cmx,o71cmx,Goodmorning….. today’s my day off and of cours...,h2w2wiy,"it doesn't work even if you try to avoid ""sayi...",target:\n\nGoodmorning….. today’s my day off a...,2.455078,...,3.045710,2.203780,3.692162,3.586367,2.167417,2.055283,3.313103,3.516466,4.003678,3.720733
1,1,2428,BreakUp,t3_o4l9ct,o4l9ct,"One moment you’re fine and the next, you’re hi...",h2i4o81,Feel them.\n\nAccept the feelings. They’re val...,target:\n\nOne moment you’re fine and the next...,2.455078,...,3.134486,3.106115,2.981231,3.701038,3.169661,1.448074,2.586455,3.220419,3.749599,2.769256
2,2,2623,depression,t3_o2qvme,o2qvme,My grandmother died a few days ago and it's so...,h288ita,"Was she ill for a long time? If so, you may ha...",target:\n\nMy grandmother died a few days ago ...,2.455078,...,2.863941,2.172391,3.714714,3.433830,2.031414,2.003925,3.296677,3.567812,3.904866,3.903016
3,3,2419,BreakUp,t3_o4r47z,o4r47z,"Sometimes I just wanna tell him everything, al...",h2ky4ze,Ouch. I’m sorry you are going through this. I’...,target:\n\nSometimes I just wanna tell him eve...,2.455078,...,2.567440,2.465548,2.929438,3.384330,2.581440,1.781015,3.079610,2.947374,3.253726,3.955260
4,4,1442,BreakUps,t3_o6sozq,o6sozq,My ex did a lot of things during our break up ...,h2v4xdp,Try to find peace in knowing that you did your...,target:\n\nMy ex did a lot of things during ou...,2.455078,...,2.764821,3.187980,3.431133,4.108836,2.323183,2.817302,3.687823,3.408082,4.227792,5.179562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,440,2434,widowers,t3_o4jpgr,o4jpgr,I have a 7 year old daughter and a 9 year old ...,h2i6lk7,"Yes it’s very normal, and expect the different...",target:\n\nI have a 7 year old daughter and a ...,2.455078,...,2.900898,2.003580,3.723593,2.870662,2.248575,1.317254,3.044449,3.529205,3.762491,3.176034
441,441,2622,MomForAMinute,t3_o2riru,o2riru,My IRL mom passed away 4 years ago when I was ...,h282ky0,"Hugs to you bro. Really, really sorry for the ...",target:\n\nMy IRL mom passed away 4 years ago ...,2.455078,...,2.432163,1.960563,3.277681,2.619379,2.383837,2.094415,2.538582,3.206351,3.226370,2.708629
442,442,2458,depression,t3_o45gcs,o45gcs,Father's day always depresses me. Its been mo...,h2ge96d,"Hey man, it seems that this holiday reminds yo...",target:\n\nFather's day always depresses me. ...,2.455078,...,3.428785,3.066746,2.749891,3.763857,2.551702,1.993464,2.578016,3.016257,3.629204,3.159668
443,443,2638,BreakUps,t3_o2k34k,o2k34k,How am I supposed to break up with him when I ...,h271ewj,You are not responsible for his feelings. Actu...,target:\n\nHow am I supposed to break up with ...,2.455078,...,2.992090,2.197164,3.573246,3.747032,2.123395,1.273673,3.372128,3.530122,3.959467,3.490994
